In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 

In [2]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from time import sleep
from winsound import Beep
from pathlib import Path
import yaml

In [3]:
def write_yaml(file_path: str, data: object):
    """
    Write data to a YAML file.

    Args:
        file_path (str): Path of the YAML file.
        data (object): Data to write.

    Returns:
        bool: True if the file was written successfully, False otherwise.
    """
    try:
        with open(Path(file_path), "w") as f:
            yaml.dump(data, f)
        print("YAML file write: OK")
        return True
    except (FileNotFoundError, IOError, yaml.YAMLError) as err:
        print(f"Failed to write YAML file {file_path}: {err}")
        return False

In [4]:
def bee(r = 3, f = 2500, d = 1000, p = 1):
    for n in range(0,r):
        Beep(f, d)
        sleep(p)


In [5]:
parameter = {}
score = {}

In [6]:
heavy_df = pd.read_csv("../data/processed/heavy.csv", index_col="Unnamed: 0")

In [7]:
heavy_df.columns

Index(['score', 'read', 'pending', 'following', 'favorite', 'have',
       'abandoned', 'Magia', 'Mecha', 'Demonios', 'Género Bender',
       'Realidad Virtual', 'Drama', 'Niños', 'Guerra', 'Harem', 'Vampiros',
       'Horror', 'Acción', 'Realidad', 'Traps', 'Militar', 'Crimen',
       'Recuentos de la vida', 'Apocalíptico', 'Psicológico', 'Misterio',
       'Musica', 'Extranjero', 'Samurái', 'Girls Love', 'Telenovela',
       'Policiaco', 'Animación', 'Parodia', 'Deporte', 'Supervivencia',
       'Aventura', 'Oeste', 'Superpoderes', 'Ecchi', 'Tragedia', 'Fantasia',
       'Gore', 'Boys Love', 'Reencarnación', 'Sobrenatural', 'Vida Escolar',
       'Historia', 'Romance', 'Ciencia Ficción', 'Thriller', 'Ciberpunk',
       'Artes Marciales', 'Comedia', 'Familia', 'chapters_count', 'type_score',
       'demography_score', 'book_status_score'],
      dtype='object')

In [8]:
#
#
#

In [9]:
X = heavy_df.drop("score", axis=1)
y = heavy_df["score"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [11]:
# Creo el scaler con los datos de train
scal = StandardScaler() # Declaro el scaler
scal.fit(X_train) # Lo "entreno". Calculo su media y std para cada feature
X_train_scal = scal.transform(X_train) # Aplico el scaler y sobreescribo los datos de train

# Aplico el mismo scaler con los datos de test
X_test_scal = scal.transform(X_test)

# Si quiero recuperar la anterior escala
#X_train = scal.inverse_transform(X_train)
#print(X_train[0])

In [12]:
poly_feats = PolynomialFeatures(degree = 2)
poly_feats.fit(X)
X_poly = poly_feats.transform(X)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [13]:
parameter["plr"] = {"poli__degree": 2}

In [14]:
predictions = lin_reg.predict(X_test)

In [15]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = lin_reg.score(X_train, y_train)
R2_Test = lin_reg.score(X_test, y_test)

In [16]:
score["plr"] = {"MAE": MAE, "MAPE": MAPE, "MSE": MSE, "RMSE": RMSE, "R2_Train": R2_Train, "R2_Test": R2_Test}

In [17]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 2.8830125049535185
MAPE: 5922824590671005.0
MSE: 11.812982560887304
RMSE: 3.4370019727790826
R2_Train: 0.20923054619125203
R2_Test: 0.2200278485169882


In [47]:
model = RandomForestRegressor(random_state=10)

parameters = {"n_estimators":[150,200],
            "max_depth":[13],
            "min_samples_leaf": [2,3,4],
            
}

# "criterion": ['absolute_error'],
# "min_samples_split": [2,5,7],
# "max_features": [6]

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

In [48]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=10), n_jobs=-1,
             param_grid={'max_depth': [13], 'min_samples_leaf': [2, 3, 4],
                         'n_estimators': [150, 200]},
             scoring='neg_mean_absolute_error')

In [52]:
best_params = {}
[best_params.update({f"regressor__{key}": value}) for key, value in dtr_gs.best_params_.items()]
parameter["rfr"] = best_params

In [21]:
feature_importances = dtr_gs.best_estimator_.feature_importances_
feature_importances_dict = dict(zip(X.columns, feature_importances))
feature_importances_sort = sorted(feature_importances_dict.items(), reverse=True)

In [22]:
feature_importances_sort

[('type_score', 0.03209906714998445),
 ('read', 0.1512478773110597),
 ('pending', 0.1138948926124426),
 ('have', 0.09030838348418381),
 ('following', 0.07394077994015849),
 ('favorite', 0.09331677741267623),
 ('demography_score', 0.0283187894568404),
 ('chapters_count', 0.08856421647000907),
 ('book_status_score', 0.15913169703518404),
 ('abandoned', 0.05283219872080116),
 ('Vida Escolar', 0.00879523447822734),
 ('Vampiros', 0.0010116371784686302),
 ('Traps', 7.180066592349279e-05),
 ('Tragedia', 0.0024466332409288074),
 ('Thriller', 0.0007581453897947773),
 ('Telenovela', 0.00010575245593711348),
 ('Supervivencia', 0.0011461384085690648),
 ('Superpoderes', 0.0015124077736891467),
 ('Sobrenatural', 0.00595132056476401),
 ('Samurái', 0.0001002011820191809),
 ('Romance', 0.006398281429488412),
 ('Reencarnación', 0.0034901920228262986),
 ('Recuentos de la vida', 0.005428720779905607),
 ('Realidad Virtual', 0.0005324756956354636),
 ('Realidad', 0.001048706559583344),
 ('Psicológico', 0.002

In [49]:
predictions = dtr_gs.predict(X_test)

In [50]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [53]:
score["rfr"] = {"MAE": MAE, "MAPE": MAPE, "MSE": MSE, "RMSE": RMSE, "R2_Train": R2_Train, "R2_Test": R2_Test}

In [51]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 1.9505762879326645
MAPE: 3743985841244049.0
MSE: 7.622585499354415
RMSE: 2.7609030224465356
R2_Train: 0.7702008559353023
R2_Test: 0.4967059012277001


R2_Train: 0.7458536102091899
R2_Test: 0.4801538640647933

In [27]:
model = ElasticNet()

parameters = {"alpha": [0.1, 0.5, 1.0],
            "l1_ratio": [0.3, 0.5, 0.7, 0.9],
            "max_iter": [1000, 2000]
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

In [28]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0],
                         'l1_ratio': [0.3, 0.5, 0.7, 0.9],
                         'max_iter': [1000, 2000]},
             scoring='neg_mean_absolute_error')

In [29]:
best_params = {}
[best_params.update({f"regressor__{key}": value}) for key, value in dtr_gs.best_params_.items()]
parameter["en"] = best_params

In [30]:
predictions = dtr_gs.predict(X_test)

In [31]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test, y_test)

In [32]:
score["en"] = {"MAE": MAE, "MAPE": MAPE, "MSE": MSE, "RMSE": RMSE, "R2_Train": R2_Train, "R2_Test": R2_Test}

In [33]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 2.9736353787984493
MAPE: 6233272402758712.0
MSE: 12.091615389253688
RMSE: 3.4773000142716604
R2_Train: 0.1907548178871653
R2_Test: 0.20163064480534565


In [34]:
model = SVR()

parameters = {
            "kernel": ["rbf", "poly"],
            "C": [0.5, 1, 2],
            "epsilon": [0.01 ,0.1, 0.3],
}
# ["rbf", "linear", "poly"]
# "C": [0.1],
# "epsilon": [0.01],

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)

In [35]:
dtr_gs.fit(X_train_scal, y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.5, 1, 2], 'epsilon': [0.01, 0.1, 0.3],
                         'kernel': ['rbf', 'poly']},
             scoring='neg_mean_absolute_error')

In [36]:
best_params = {}
[best_params.update({f"regressor__{key}": value}) for key, value in dtr_gs.best_params_.items()]
parameter["svr"] = best_params

In [37]:
dtr_gs.best_params_

{'C': 2, 'epsilon': 0.01, 'kernel': 'rbf'}

In [38]:
predictions = dtr_gs.predict(X_test_scal)

In [39]:
MAE = mean_absolute_error(y_test, predictions)
MAPE = mean_absolute_percentage_error(y_test, predictions)
MSE = mean_squared_error(y_test, predictions)
RMSE = np.sqrt(mean_squared_error(y_test, predictions))
R2_Train = dtr_gs.best_estimator_.score(X_train_scal, y_train)
R2_Test = dtr_gs.best_estimator_.score(X_test_scal, y_test)

In [40]:
score["svr"] = {"MAE": MAE, "MAPE": MAPE, "MSE": MSE, "RMSE": RMSE, "R2_Train": R2_Train, "R2_Test": R2_Test}

In [41]:
print("MAE:", MAE)
print("MAPE:", MAPE)
print("MSE:", MSE)
print("RMSE:", RMSE)
print("R2_Train:", R2_Train)
print("R2_Test:", R2_Test)

MAE: 2.3649876441186275
MAPE: 5512983597127248.0
MSE: 11.497231143479599
RMSE: 3.390756721364657
R2_Train: 0.3244135830508781
R2_Test: 0.24087586984436404


In [54]:
score

{'plr': {'MAE': 2.8830125049535185,
  'MAPE': 5922824590671005.0,
  'MSE': 11.812982560887304,
  'RMSE': 3.4370019727790826,
  'R2_Train': 0.20923054619125203,
  'R2_Test': 0.2200278485169882},
 'rfr': {'MAE': 1.9505762879326645,
  'MAPE': 3743985841244049.0,
  'MSE': 7.622585499354415,
  'RMSE': 2.7609030224465356,
  'R2_Train': 0.7702008559353023,
  'R2_Test': 0.4967059012277001},
 'en': {'MAE': 2.9736353787984493,
  'MAPE': 6233272402758712.0,
  'MSE': 12.091615389253688,
  'RMSE': 3.4773000142716604,
  'R2_Train': 0.1907548178871653,
  'R2_Test': 0.20163064480534565},
 'svr': {'MAE': 2.3649876441186275,
  'MAPE': 5512983597127248.0,
  'MSE': 11.497231143479599,
  'RMSE': 3.390756721364657,
  'R2_Train': 0.3244135830508781,
  'R2_Test': 0.24087586984436404}}

In [55]:
parameter

{'plr': {'poli__degree': 2},
 'rfr': {'regressor__max_depth': 13,
  'regressor__min_samples_leaf': 3,
  'regressor__n_estimators': 200},
 'en': {'regressor__alpha': 0.1,
  'regressor__l1_ratio': 0.3,
  'regressor__max_iter': 1000},
 'svr': {'regressor__C': 2,
  'regressor__epsilon': 0.01,
  'regressor__kernel': 'rbf'}}

In [56]:
write_yaml("../models/heavy/model_config.yaml", parameter)

YAML file write: OK


True

In [57]:
write_yaml("../models/heavy/score.yaml", score)

YAML file write: OK


True

In [46]:
bee()